# Parsing du document texte

In [1]:
import codecs
import pandas as pd


# Ouvrir le fichier en lecture seule
file = codecs.open('Planning_JO2024_v3.txt', "r", "utf-8")
# utiliser readlines pour lire toutes les lignes du fichier
# La variable "lignes" est une liste contenant toutes les lignes du fichier
lines = file.readlines()
# fermez le fichier après avoir lu les lignes
file.close()


words = []
for line in lines:
    words += line.split()
    
demarq1_discipline = "calendrier reste susceptible de modification jusqu'à la conclusion des Jeux Olympiques.".split()
demarq2_discipline = "A noter que l'ordre des matchs sera déterminé pendant les Jeux en fonction du temps de repos des athlètes, des demandes des diffuseurs et d'autres paramètres qu'il n'est pas possible d'anticip".split()
discipline = 'ATHLETISME'

data = []
i = 0
while i < (len(words)):
    # print("Passage de boucle")
    word = words[i]        
    if words[i].upper() == "JOUR":
        if len(words) > i + 4:
            jour = words[i+1]
            date = words[i+2] + " " + words[i+3] + " " + words[i+4]
            i +=4
            # print("jour ", jour)
        else:
            print("erreur")

    # discipline
    if words[i:i+len(demarq1_discipline)] == demarq1_discipline:
        i+=len(demarq1_discipline)
        if words[i:i+len(demarq2_discipline)] == demarq2_discipline:
            i+=len(demarq2_discipline)
        discipline = ""
        espace = ""
        while words[i] != "Calendrier":
            discipline += espace + words[i]
            espace = " "
            i += 1
            # print("discipline ", discipline)
    
    if words[i] == "Session":
        session = words[i+1]
        i += 2
        # print("session ", session)
    if words[i] == "Début":
        debut = words[i+2]
        i += 3
    if words[i] == "Fin":
        fin = words[i+2]
        i += 3
        lieu_epreuve = ""
        espace = ""
        while words[i] != "Nom":
            lieu_epreuve += espace + words[i]
            espace = " "
            i += 1
            # print('lieu ', lieu_epreuve)
    nom_epreuve = ""
    if words[i:i+4] == ['Nom', 'de', "l'épreuve/des", 'épreuves']:
        i += 4
        espace = ""
        while len(words) > i and words[i].upper() not in ["JOUR", "SESSION"] and words[i:i+2] != ['Le', 'calendrier']:
            nom_epreuve += espace + words[i]
            espace = " "
            # print(words[i])
            i += 1
        i-=1
        # if session == "BOX24":
        #     print(session, " ", nom_epreuve)
        data.append([jour, date, discipline, session, debut, fin, lieu_epreuve, nom_epreuve])
    i += 1

def traitement_discipline(string):
    if len(string) == 0:
        return ""
    else:
        return string[0].upper() + string[1:].lower()


df = pd.DataFrame(data, columns=['num_jour', 'date', 'discipline', 'num_session', 'debut_epreuve', 'fin_epreuve', 'lieu_epreuve', 'nom_epreuve'])
df['discipline'] = df['discipline'].apply(traitement_discipline)
df.loc[df['num_session'] == 'BOX24']
#df.to_csv('../data/session_planning_olympics.csv')

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve
147,11,Mardi 6 Août,Boxe,BOX24,21:30,23:30,Stade Roland-Garros - Court Philippe-Chatrier,"71kg - Hommes, demi-finale, combat 17 71kg - H..."


In [2]:
df

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve
0,6,Jeudi 1 Août,Athletisme,ATH01,07:30,11:05,Trocadéro,"20 km marche - hommes, finale 20 km marche - f..."
1,7,Vendredi 2 Août,Athletisme,ATH02,10:00,13:00,Stade de France,"Décathlon - hommes, 100 m Lancer de marteau - ..."
2,7,Vendredi 2 Août,Athletisme,ATH03,17:40,22:00,Stade de France,"Décathlon - hommes, saut en hauteur 5 000 m - ..."
3,8,Samedi 3 Août,Athletisme,ATH04,10:00,16:00,Stade de France,"Décathlon - hommes, 110 m haies Saut à la perc..."
4,8,Samedi 3 Août,Athletisme,ATH05,19:00,22:00,Stade de France,"1 500 m - hommes, Repêchages Lancer de poids -..."
...,...,...,...,...,...,...,...,...
752,14,Vendredi 9 Août,Water polo,WPO32,18:00,21:00,Paris La Défense Arena,"Hommes, match de classement Hommes, demi-finale"
753,15,Samedi 10 Août,Water polo,WPO33,09:00,12:00,Paris La Défense Arena,"Femmes, match de classement Femmes, match pour..."
754,15,Samedi 10 Août,Water polo,WPO34,14:00,21:00,Paris La Défense Arena,"Femmes, match de classement Femmes, match pour..."
755,16,Dimanche 11 Août,Water polo,WPO35,09:00,12:00,Paris La Défense Arena,"Hommes, match de classement Hommes, match pour..."


# Parsing noms d'epreuves

In [3]:
import json

# separateurs = [' - ', ', '], 
# noms = ['epreuve', 'genre', 'etape']


def parser(texte, separateurs, noms):
    #print(texte)
    epreuves = {'content' : []}
    # epreuves = []
    # epreuve, genre, etape = '', '', ''
    dico_epreuve = {}
    i, i0 = 0, 0
    while i < len(texte):
        #print(i)
        for n in range(len(separateurs)):
            separateur = separateurs[n]
            #print("Chgt separateur : ", separateur)
            while i < len(texte):
                if i + len(separateur) < len(texte) and texte[i:i+len(separateur)] == separateur:
                    dico_epreuve[noms[n]] = texte[i0:i]
                    #print(texte[i0:i])
                    i+=len(separateur)
                    i0 = i
                    break
                i+=1
        #print(i0)
        while i <= len(texte):
            if  i == len(texte) or nom_epreuve(texte, i):
                dico_epreuve[noms[-1]] = texte[i0:i]
                i0 = i
                break
            i+=1
        epreuves['content'].append(dico_epreuve.copy())#json.dumps(dico_epreuve.copy()))
        # epreuves.append(dico_epreuve.copy())
        i+=1
    return json.dumps(epreuves)

noms_sports = [
    "10 m",
    "50 m"
    "20 km",
    "100 m",
    "200 m",
    "1 500 m",
    "5 000 m",
    "800 m",
    "10 000 m",
    "3 000 m",
    "400 m",
    "110 m",
    "50kg",
    "50 kg",
    "51kg",
    "54kg",
    "60kg",
    "66kg",
    "75kg",
    "63.5kg",
    "63,5kg",
    "80kg",
    "92kg",
    "71kg",
    "57kg",
    "+92kg",
    "+67kg",
    "+80kg",
    "-67kg",
    "-80kg",
    "-57kg",
    "-68kg",
    "-49kg",
    "-58kg",
    "-48kg",
    "-60kg",
    "-66kg",
    "-52kg",
    "-57kg",
    "-73kg",
    "+78kg",
    "+100kg",
    "-81kg",
    "-90kg",
    "-78kg",
    "-63kg",
    "-70kg",
    "-100kg",
    "60kg",
    "68kg",
    "130kg",
    "77kg",
    "86kg",
    "97kg",
    "50kg",
    "74kg",
    "76kg",
    "53kg",
    "57kg",
    "65kg",
    "87kg",
    "62kg",
    "67kg",
    "125kg",
]

pas_noms_epreuve = [
    "Femmes",
    "Hommes",
    "Repêchage",
    "Qualifications",
    "Eliminatoires",
    "Quart de finale",
    "Demi-finale",
    "Match pour la médaille",
    "Série",
    "Course pour",
    "Grand",
    "Prix"
]

# inclure pas nom d'épreuve
def nom_epreuve(texte, i):
    if texte[i].isupper():
        for nom in pas_noms_epreuve:
            if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
                return False            
        if i + 3 < len(texte) and texte[i+1] != " " and texte[i+2] != " " and texte[i+3] != " " :
            return True
    for nom in noms_sports:
        if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
            return True
    return False

noms_etapes = ["eliminatoires", "repêchage"] # --> enlever majuscule

# Rajouter condition sur nom sport: plus de 3 lettres
# pour tej les "finale A/B" et autres

cles_parsers = {
    'ATH ROW BMF CTR CLB FEN ARC SHO BOX CSL CSP WRE JUD SWM TEN TTE BDM MPN DIV GLF GAR TKW' : {
            'separateurs' : [' - ', ', '], 
            'noms' : ['epreuve', 'genre', 'etape']
    },
    'SKB' : {
            'separateurs' : [' - ', ', '], 
            'noms' : ['genre', 'epreuve', 'etape']
    },
    'BKB BMX BK3 RU7 BKG GTR HBL HOC SRF VVO WPO' : {
        'separateurs' : [', '], 
        'noms' : ['genre', 'etape']
    },
    'CRD OWS MTB WLF' : {
        'separateurs' : [' - '], 
        'noms' : ['epreuve', 'genre']
    },
    'SAL' : {
        'separateurs' : [' - '], 
        'noms' : ['epreuve', 'etape']
    },
    'FBL' : {
        'separateurs' : [', ', ' - '], 
        'noms' : ['genre', 'etape', 'epreuve']
    },
    'VBV' : {
        'separateurs' : [' - '], 
        'noms' : ['etape', 'genre']
    },
    'GRY' : {
        'separateurs' : [', '], 
        'noms' : ['epreuve', 'etape']
    },
    'SWA EQU' : {
        'separateurs' : [], 
        'noms' : ['epreuve']
    },
    'WLF TRI' : {
        'separateurs' : [], 
        'noms' : ['genre']
    },
}


def match_cle_parser(code_disc, cles_parsers):
    for cle in cles_parsers.keys():
        if code_disc in cle:
            return cles_parsers[cle]
    print(f"Cette discipline n'est pas prise en charge : {code_disc}")

def parsing_epreuve(row):
    texte = row.nom_epreuve
    if len(str(row.num_session)) > 2:
        disc = str(row.num_session)[:3]
        cle_parser = match_cle_parser(disc, cles_parsers)
        if cle_parser is not None:
            return parser(texte, cle_parser['separateurs'], cle_parser['noms'])

# disc_analysees = ['ATHLETISME', 'AVIRON', 'BASKETBALL', 'BASKETBALL 3x3', 'BMX FREESTYLE', 'BMX RACING', 'Breaking']

df['parsing_epreuve'] = df.apply(parsing_epreuve, axis=1)
#df.to_csv('../data/session_planning_olympics_v10.csv')
# df

## Matching localisations

In [4]:
pd.read_csv("matching_infras4.csv")

,Planning,Infra
0,Trocadéro,Trocadéro
1,Stade de France,Stade de France
2,Hôtel de Ville (site de départ) - Invalides (s...,Hôtel de ville de Paris
3,Stade nautique de Vaires-sur-Marne - Bassin ea...,Stade nautique de Vaires-sur-Marne - Bassin ea...
4,Arena Porte de La Chapelle,Arena La Chapelle
5,Stade Pierre Mauroy*,Stade Pierre Mauroy
6,Arena Bercy,Arena Bercy
7,La Concorde 1,Stade de la Concorde
8,La Concorde,Stade de la Concorde
9,Stade BMX de Saint-Quentin-en-Yvelines,Stade BMX de Saint-Quentin-en-Yvelines


In [5]:
import pandas as pd

# import data
df1 = pd.read_csv("paris-2024-sites-olympiques-et-paralympiques-franciliens.csv", delimiter=";")
# df = pd.read_csv("../data/session_planning_olympics_v9.csv")
matching_infras = pd.read_csv("matching_infras4.csv")
matching_infras2 = matching_infras.rename(columns={"Planning" : "lieu_epreuve", "Infra" : "NOM"})

# merge
df3 = pd.merge(df, matching_infras2, how='left', on="lieu_epreuve")
df4 = pd.merge(df3, df1, how='left', on="NOM")

# processing localisation
df4.loc[df4['Geo Point'].notna(), 'latitude'] = df4.loc[df4['Geo Point'].notna(), 'Geo Point'].apply(lambda x: float(x.split(', ')[0]))
df4.loc[df4['Geo Point'].notna(), 'longitude'] = df4.loc[df4['Geo Point'].notna(), 'Geo Point'].apply(lambda x: float(x.split(', ')[1]))


# process dates
df4.loc[df4['date'] == 'dimanche 28 J', 'date'] = 'Dimanche 28 Juillet'
df4.loc[df4['date'] == 'Lundi 29 J', 'date'] = 'Lundi 29 Juillet'
df4.loc[df4['date'] == 'dimanche 11 Août', 'date'] = 'Dimanche 11 Août'
df4.loc[df4['date'] == 'Dim 11 Août', 'date'] = 'Dimanche 11 Août'
df4.loc[df4['date'] == 'Dim 4 Août', 'date'] = 'Dimanche 4 Août'
df4.loc[df4['date'] == 'Dim 28 Juillet', 'date'] = 'Dimanche 28 Juillet'
df4.loc[df4['date'] == 'dimanche 4 Août', 'date'] = 'Dimanche 4 Août'
df4.loc[df4['date'] == 'dimanche 28 Juillet', 'date'] = 'Dimanche 28 Juillet'

# filtrer les nan
print(len(df4.loc[df4['Geo Point'].notna()])/len(df4)) # --> on garde 80% des infras 

# df4 = df4.loc[df4['Geo Point'].notna()]
# df4.to_csv('../data/session_planning_pars_with_loc_v14.csv')

1.0


In [6]:
print(df.loc[df["num_session"] == "BOX24", 'num_session'])
print(df3.loc[df3["num_session"] == "BOX24", 'num_session'])

print(df4.loc[df4["num_session"] == "BOX24", 'num_session'])

147    BOX24
Name: num_session, dtype: object
147    BOX24
Name: num_session, dtype: object
174    BOX24
Name: num_session, dtype: object


## Mapping des liens url

In [7]:
url_root = "https://www.paris2024.org/fr/sport/"
df4= df4.assign(url= url_root + df4['discipline'] + "/")
df4.url = df4.url.str.replace(" ", "-")
df4.url = df4.url.str.replace("Canoë,-course-en-ligne", "canoe-sprint")
df4.url = df4.url.str.replace("Gymnastique-trampoline", "trampoline")
df4.url.unique()

#Tous les liens ont été vérifiés manuellement, ils sont accesibles.

array(['https://www.paris2024.org/fr/sport/Athletisme/',
       'https://www.paris2024.org/fr/sport/Aviron/',
       'https://www.paris2024.org/fr/sport/Badminton/',
       'https://www.paris2024.org/fr/sport/Basketball/',
       'https://www.paris2024.org/fr/sport/Basketball-3x3/',
       'https://www.paris2024.org/fr/sport/Bmx-freestyle/',
       'https://www.paris2024.org/fr/sport/Bmx-racing/',
       'https://www.paris2024.org/fr/sport/Breaking/',
       'https://www.paris2024.org/fr/sport/Boxe/',
       'https://www.paris2024.org/fr/sport/Canoe-slalom/',
       'https://www.paris2024.org/fr/sport/canoe-sprint/',
       'https://www.paris2024.org/fr/sport/Cyclisme-sur-piste/',
       'https://www.paris2024.org/fr/sport/Cyclisme-sur-route/',
       'https://www.paris2024.org/fr/sport/Escalade/',
       'https://www.paris2024.org/fr/sport/Escrime/',
       'https://www.paris2024.org/fr/sport/Football/',
       'https://www.paris2024.org/fr/sport/Golf/',
       'https://www.paris2024.

### Create csv with wub urls

In [8]:
df4.to_csv('../output.csv')

### PA ajoute les capacités

In [9]:
# ajout des capacités
df4.head(1)

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve,parsing_epreuve,NOM,Geo Point,SITES OLYMPIQUES PARALYMPIQUES,SPORTS,latitude,longitude,url
0,6,Jeudi 1 Août,Athletisme,ATH01,07:30,11:05,Trocadéro,"20 km marche - hommes, finale 20 km marche - f...","{""content"": [{""epreuve"": ""20 km marche"", ""genr...",Trocadéro,"48.863045, 2.2871938",NaN,NaN,48.863045,2.287194,https://www.paris2024.org/fr/sport/Athletisme/


In [10]:
stade_vers_capacite = {
    'Stade de France': 80000,
    'Stade nautique de Vaires-sur-Marne - Bassin eaux calmes': 14000,
    'Arena Porte de La Chapelle': 9000,
    'Stade BMX de Saint-Quentin-en-Yvelines': 3000,
    'Stade nautique de Vaires-sur-Marne - Bassin eaux vives': 24000,
    'Vélodrome National de Saint-Quentin-en-Yvelines': 6000,
    'Grand Palais': 8000,
    'Parc des Princes'   : 48583,
    'Arena Bercy': 20300,
    'Porte de La Chapelle Arena': 9000,
    'Arena Paris Sud 6': 8000,
    'Stade Yves-du-Manoir - Terrain 2': 13500,
    'Stade Yves-du-Manoir - Terrain 1': 13500,
    'Arena Champ-de-Mars': 8000,
    'Paris La Défense Arena': 17000,
    'Arena Paris Nord': 8000,
    'Château de Versailles': 40000,
    'Centre aquatique': 2500,
    'Invalides': 8000, #17000
    'Stade Tour Eiffel': 12860,
    # add new capacite 
    'Arena Paris Sud 1': 8000,
    'Arena Paris Sud 4': 8000,
    'La Concorde 1': 35000,
    'Stade Roland-Garros - Court Philippe-Chatrier': 14911,
    'Golf National': 32720,
    'Stade Roland-Garros - Court Suzanne-Lenglen': 10056,
    "Site d'escalade du Bourget": 6000,
    'La Concorde 3': 35000,
    'Stade Roland-Garros - Court Simonne-Mathieu': 5000,
    'Stade Roland-Garros - Courts Annexes': 2400,
    "Colline d'Elancourt": 15000,
    "Hôtel de Ville (site de départ) - Invalides (site d'arrivée)": 8000,
    'La Concorde': 35000,
    'La Concorde 4': 35000,
    'Départ Invalides - Arrivée Pont Alexandre III': 8000,
    'Trocadéro' : 3349,
    'Stade Pierre Mauroy*' : 50000,
    'Stade Geoffroy-Guichard' : 42000,
    'Stade de la Beaujoire' : 35000,
    'Stade de Nice' : 36000,
    'Stade de Bordeaux' : 42000,
    'Stade de Lyon' : 60000,
    'Stade de Marseille' : 67000,
    'Stade Pierre Mauroy' : 50000,
    'Pont Alexandre III' : 1000,
    'Teahupoo' : 600,
    'Centre National de Tir de Châteauroux*' : 3500       
}

In [11]:
df4.loc[df4['latitude'] == 48.862286, 'lieu_epreuve']

202    Départ Invalides - Arrivée Pont Alexandre III
616                                        Invalides
617                                        Invalides
618                                        Invalides
619                                        Invalides
620                                        Invalides
621                                        Invalides
622                                        Invalides
623                                        Invalides
624                                        Invalides
625                                        Invalides
626                                        Invalides
627                                        Invalides
628                                        Invalides
629                                        Invalides
630                                        Invalides
631                                        Invalides
632                                        Invalides
633                                        Inv

In [12]:
from tqdm import tqdm

for stade in tqdm(stade_vers_capacite.keys()):
    df4.loc[df4.lieu_epreuve == stade, 'capacite'] = stade_vers_capacite[stade]

100%|██████████| 47/47 [00:00<00:00, 977.21it/s]


In [13]:
df4.loc[df4.capacite.isna(), 'lieu_epreuve'].unique()

array([], dtype=object)

In [14]:
df4.capacite.isna().sum()
# toutes les capacités sont mappées

0

In [15]:
df4.head(1)

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve,parsing_epreuve,NOM,Geo Point,SITES OLYMPIQUES PARALYMPIQUES,SPORTS,latitude,longitude,url,capacite
0,6,Jeudi 1 Août,Athletisme,ATH01,07:30,11:05,Trocadéro,"20 km marche - hommes, finale 20 km marche - f...","{""content"": [{""epreuve"": ""20 km marche"", ""genr...",Trocadéro,"48.863045, 2.2871938",NaN,NaN,48.863045,2.287194,https://www.paris2024.org/fr/sport/Athletisme/,3349.0


In [16]:
df4.to_csv('../output.csv', index=False)

## Autres manips

In [17]:
df4.loc[1, :]

num_jour                                                                          7
date                                                                Vendredi 2 Août
discipline                                                               Athletisme
num_session                                                                   ATH02
debut_epreuve                                                                 10:00
fin_epreuve                                                                   13:00
lieu_epreuve                                                        Stade de France
nom_epreuve                       Décathlon - hommes, 100 m Lancer de marteau - ...
parsing_epreuve                   {"content": [{"epreuve": "D\u00e9cathlon", "ge...
NOM                                                                 Stade de France
Geo Point                                                       48.924388, 2.359871
SITES OLYMPIQUES PARALYMPIQUES                     Site olympique,Site paral

In [18]:
df4['date'].unique()

array(['Jeudi 1 Août', 'Vendredi 2 Août', 'Samedi 3 Août',
       'Dimanche 4 Août', 'Lundi 5 Août', 'Mardi 6 Août',
       'Mercredi 7 Août', 'Jeudi 8 Août', 'Vendredi 9 Août',
       'Samedi 10 Août', 'Samedi 27 Juillet', 'Dimanche 28 Juillet',
       'Lundi 29 Juillet', 'Mardi 30 Juillet', 'Mercredi 31 Juillet',
       'Dimanche 11 Août', 'Mercredi 24 Juillet', 'Jeudi 25 Juillet'],
      dtype=object)

In [19]:
df4.loc[df4['date'] == 'Skeet - hommes,']

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve,parsing_epreuve,NOM,Geo Point,SITES OLYMPIQUES PARALYMPIQUES,SPORTS,latitude,longitude,url,capacite


In [20]:
for i in range(1, len(df4)):
    print(df4.loc[i, 'discipline'], " : ", df4.loc[i, 'parsing_epreuve'])

Athletisme  :  {"content": [{"epreuve": "D\u00e9cathlon", "genre": "hommes", "etape": ""}, {"epreuve": "100 m Lancer de marteau", "genre": "hommes", "etape": "qualifications "}, {"epreuve": "Saut en hauteur", "genre": "femmes", "etape": "qualifications "}, {"epreuve": "100 m", "genre": "femmes", "etape": "tour pr\u00e9liminaire "}, {"epreuve": "D\u00e9cathlon", "genre": "saut en longueur", "etape": "hommes "}, {"epreuve": "1 500 m", "genre": "hommes", "etape": "1er tour "}, {"epreuve": "100 m", "genre": "femmes", "etape": "1er tour "}, {"epreuve": "D\u00e9cathlon", "genre": "hommes", "etape": "lancer de poids"}]}
Athletisme  :  {"content": [{"epreuve": "D\u00e9cathlon", "genre": "hommes", "etape": "saut en hauteur "}, {"epreuve": "5 000 m", "genre": "femmes", "etape": "1er tour "}, {"epreuve": "Triple saut", "genre": "femmes", "etape": "qualifications "}, {"epreuve": "Lancer de disque", "genre": "femmes", "etape": "qualifications "}, {"epreuve": "Relais 4 x 400 m", "genre": "mixte", "e

In [21]:
#Reste plus que le matching --> NAAN
matching_infras2

,lieu_epreuve,NOM
0,Trocadéro,Trocadéro
1,Stade de France,Stade de France
2,Hôtel de Ville (site de départ) - Invalides (s...,Hôtel de ville de Paris
3,Stade nautique de Vaires-sur-Marne - Bassin ea...,Stade nautique de Vaires-sur-Marne - Bassin ea...
4,Arena Porte de La Chapelle,Arena La Chapelle
5,Stade Pierre Mauroy*,Stade Pierre Mauroy
6,Arena Bercy,Arena Bercy
7,La Concorde 1,Stade de la Concorde
8,La Concorde,Stade de la Concorde
9,Stade BMX de Saint-Quentin-en-Yvelines,Stade BMX de Saint-Quentin-en-Yvelines
